# gfe-db / Build graph

This notebook contains scripts for an update pipeline to [`nmdp-bioinformatics/gfe-db`](https://github.com/nmdp-bioinformatics/gfe-db). When run the notebook creates flat CSV files that can be read and parsed by Cypher's `LOAD CSV` clause to either create new nodes and relationships or merge existing ones.

#### Key Columns
Relationships are defined by a key columns in each CSV:
- `(:GFE)-[:HAS_FEATURE]->(:FEATURE)` on `hla_name` (or `alleleId`)
- `(:GFE)-[:HAS_SEQUENCE]->(:SEQUENCE)` on `sequence`
- `(:GFE)-[:HAS_ALIGNMENT]->(:SEQUENCE)` on `sequence`
- `(:GFE)-[:HAS_ALIGNMENT]->(:GEN_ALIGN)`, `(:NUC_ALIGN)`, `(:PROT_ALIGN)` on `a_name`
- `(:SEQUENCE)-[:HAS_CDS]->(:CDS)` on `alleleId`
- `(:IMGT_HLA)-[:HAS_GFE]->(:GFE)` on `hla_name`
- `(:IMGT_HLA)-[:HAS_FEATURE]->(:FEATURE)` on `hla_name`
- `(:IMGT_HLA)-[:HAS_SEQUENCE]->(:SEQUENCE)` on `alleleId`
- `(:IMGT_HLA)-[:HAS_ALIGNMENT]->(:SEQUENCE)` on `alleleId`
- `(:IMGT_HLA)-[:HAS_ALIGNMENT]->(:GEN_ALIGN)`, `(:NUC_ALIGN)`, `(:PROT_ALIGN)` on `hla_name`
- `(:IMGT_HLA)<-[:G]-(:G)` on `hla_name`
- `(:IMGT_HLA)<-[:lg]-(:lg)` on `hla_name`
- `(:IMGT_HLA)<-[:lgx]-(:lgx)` on `hla_name`
- `(:IMGT_HLA)<-[:2nd_FIELD]-(:2nd_FIELD)` on `hla_name`

# Libraries

In [1]:
import os
import sys
sys.path[0] = '../'
import logging

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from bin.build_gfedb import *

# Environment

In [2]:
imgt_hla = 'https://www.ebi.ac.uk/ipd/imgt/hla/docs/release.html'
imgt_hla_media_url = 'https://media.githubusercontent.com/media/ANHIG/IMGTHLA/'
imgt_hla_raw_url = 'https://raw.githubusercontent.com/ANHIG/IMGTHLA/'

imgt_kir = 'https://www.ebi.ac.uk/ipd/kir/docs/version.html'
kir_url = 'ftp://ftp.ebi.ac.uk/pub/databases/ipd/kir/KIR.dat'


data_dir = "../../data/" #os.path.dirname(__file__) + "/../../data/"

expre_chars = ['N', 'Q', 'L', 'S']

In [3]:
lastseqid = 1
lastid = 1
lastcdsid = 1

seqids = {}
cdsids = {}
alleleids = {}
group_edges = {}
trans_edges = {}

# The alleles are removed when the allele_nodes.csv is built
skip_alleles = ["HLA-DRB5*01:11", "HLA-DRB5*01:12", "HLA-DRB5*01:13",
                "HLA-DRB5*02:03", "HLA-DRB5*02:04", "HLA-DRB5*02:05",
                "HLA-DRB5*01:01:02", "HLA-DRB5*01:03", "HLA-DRB5*01:05",
                "HLA-DRB5*01:06", "HLA-DRB5*01:07", "HLA-DRB5*01:09",
                "HLA-DRB5*01:10N", "HLA-C*05:208N", "HLA-C*05:206"]

hla_loci = ['HLA-A', 'HLA-B', 'HLA-C', 'HLA-DRB1', 'HLA-DQB1',
            'HLA-DPB1', 'HLA-DQA1', 'HLA-DPA1', 'HLA-DRB3',
            'HLA-DRB4', 'HLA-DRB5']

hla_align = ['HLA-A', 'HLA-B', 'HLA-C', 'HLA-DRB1', 'HLA-DQB1',
             'HLA-DPB1', 'HLA-DQA1', 'HLA-DPA1']

kir_loci = ["KIR3DS1", "KIR3DP1", "KIR3DL3", "KIR3DL2", "KIR3DL1",
            "KIR2DS5", "KIR2DS4", "KIR2DS3", "KIR2DS2", "KIR2DS1",
            "KIR2DP1", "KIR2DL5B", "KIR2DL5A", "KIR2DL4"]

kir_aligloci = ["KIR2DL4", "KIR2DP1", "KIR2DS1", "KIR2DS2", "KIR2DS3",
                "KIR2DS4", "KIR2DS5", "KIR3DL1", "KIR3DL2", "KIR3DL3",
                "KIR3DP1"]

ard_groups = ['G', 'lg', 'lgx']

align = True

In [4]:
kir = None

if kir:
    load_loci = hla_loci + kir_loci
else:
    load_loci = hla_loci

from seqann import gfe
gfe_maker = gfe.GFE(verbose=True, verbosity=2,
                load_features=False, store_features=True,
                loci=load_loci)

In [5]:
dbversions = ["3360"]

verbose = True

In [6]:
out_dir = '' #args.out_dir
release_n = 1 #args.number
releases = '3360'#args.releases
verbosity = 1

align = True
kir = True
debug = False
verbose = True

#if args.kir:
kir = True

#if args.align:
align = True

#if args.verbose:
verbose = True

if kir:
    load_loci = hla_loci + kir_loci
else:
    load_loci = hla_loci

#if args.debug:
#    logging.info("Running in debug mode")
#    load_loci = ["HLA-A"]
#    kir = False
#    debug = True
#    verbose = True
#    verbosity = 2
#    release_n = 1

gfe_e = []
seq_e = []
seq_n = []
cds_n = []
grp_e = []
trs_e = []
allele_n = []

# Get last five IMGT/HLA releases
if releases:
    dbversions = [db for db in releases.split(",")]
else:
    dbversions = pd.read_html(imgt_hla)[0]['Release'][0:release_n].tolist()

# Get latest IMGT/KIR release
kir_release = pd.read_html(imgt_kir)[0]['Release'][0]

from seqann import gfe
gfe_maker = gfe.GFE(verbose=verbose, verbosity=verbosity,
                    load_features=False, store_features=True,
                    loci=load_loci)

## Helper Functions

In [7]:
def get_cds(allele):

    feat_types = [f.type for f in allele.features]

    if "CDS" in feat_types:
        cds_features = allele.features[feat_types.index("CDS")]
        if 'translation' in cds_features.qualifiers:

            if cds_features.location is None:
                logger.info("No CDS location for feature in allele: ", allele.name)
            else:
                bp_seq = str(cds_features.extract(allele.seq))
                aa_seq = cds_features.qualifiers['translation'][0]
                
    return bp_seq, aa_seq

# Processes

## HLA process

Code in this process contains variables that define nodes.

In [8]:
#dbversions = ["3410", "3420", "3430"]
dbversions = ["3360"]

In [19]:
import ast

# Loop through DB versions
for dbversion in dbversions:
    
    imgt_release = f'{dbversion[0]}.{dbversion[1:3]}.{dbversion[3]}'

    db_striped = ''.join(dbversion.split("."))

    if align:
        gen_aln, nuc_aln, prot_aln = hla_alignments(db_striped)

    ard = ARD(db_striped)

    # The github URL changed from 3350 to media
    if int(db_striped) < 3350:
        dat_url = imgt_hla_raw_url + db_striped + '/hla.dat'
    else:
        dat_url = imgt_hla_media_url + db_striped + '/hla.dat'

    dat_file = data_dir + 'hla.' + db_striped + ".dat"

    # Downloading DAT file
    if not os.path.isfile(dat_file):
        if verbose:
            logging.info("Downloading dat file from " + dat_url)
        urllib.request.urlretrieve(dat_url, dat_file)

    # Parse DAT file
    a_gen = SeqIO.parse(dat_file, "imgt")

    if verbose:
        logging.info("Finished parsing dat file")

    i = 0

    ### Initialize lists for CSV output (input to LOAD CSV in Neo4j)
    # Lists contain unique dicts and are converted to dataframes, then output to CSV for Neo4j import
    gfe_sequences = []
    gen_alignments = []
    nuc_alignments = []
    prot_alignments = []
    all_features = []
    all_groups = []
    all_cds = []
    ###
    
    for idx, allele in enumerate(a_gen):

        if hasattr(allele, 'seq'):
            hla_name = allele.description.split(",")[0]
            loc = allele.description.split(",")[0].split("*")[0]

            if hla_name in skip_alleles:
                logging.info(
                    "SKIPPING = " + allele.description.split(",")[0] + " " + dbversion)
                continue

            if debug and (loc != "HLA-A" and i > 20):
                continue

            if (loc in hla_loci or loc == "DRB5") and (len(str(allele.seq)) > 5):
                if debug:
                    logging.info(
                        "HLA = " + allele.description.split(",")[0] + " " + dbversion)

                a_name = allele.description.split(",")[0].split("-")[1]
                groups = [["HLA-" + ard.redux(a_name, grp), grp] if ard.redux(a_name, grp) != a_name else None for
                          grp in ard_groups]
                seco = [[to_second(a_name), "2nd_FIELD"]]
                groups = list(filter(None, groups)) + seco
                complete_annotation = get_features(allele)
                ann = Annotation(annotation=complete_annotation,
                                 method='match',
                                 complete_annotation=True)

                # This process takes a long time
                features, gfe = gfe_maker.get_gfe(ann, loc)

                # gen_aln, nuc_aln, prot_aln
                aligned_gen = ''
                aligned_nuc = ''
                aligned_prot = ''

                if align:
                    if allele.description.split(",")[0] in gen_aln[loc]:
                        aligned_gen = gen_aln[loc][allele.description.split(",")[
                            0]]

                    if allele.description.split(",")[0] in nuc_aln[loc]:
                        aligned_nuc = nuc_aln[loc][allele.description.split(",")[
                            0]]

                    if allele.description.split(",")[0] in prot_aln[loc]:
                        aligned_prot = prot_aln[loc][allele.description.split(",")[
                            0]]
            
            ### Build dicts describing nodes and edges for each allele
            
            # Notes:
            # all edges are joined using foreign keys:
            #  - GFE --> SEQUENCE on alleleId
            #  - GFE --> [GEN_ALIGN, NUC_ALIGN, PROT_ALIGN] on a_name
            #  - GFE --> FEATURE on alleleId (or hla_name)
            # "alleleId" is assigned allele.id value
            # "sequenceId" is replaced with alleleId
            # feature key "name" is now "term"
            
            # Questions:
            # - Should GFE name be assigned to each node?

            # Separate CSV file
            gfe_sequence = {
                "alleleId": allele.id,
                "gfe_name": gfe,
                "locus": loc,
                "hla_name": hla_name,
                "a_name": a_name, # hla_name.split("-")[1]
                "sequence": str(allele.seq),
                "length": len(str(allele.seq)),
                "imgt_release": imgt_release
            }

            # Separate CSV file, GFE foreign key: a_name
            gen_alignment = {
                "label": "GEN_ALIGN",
                "hla_name": hla_name,
                "a_name": a_name, # hla_name.split("-")[1]
                "length": len(aligned_gen),
                "rank": "0", # TO DO: confirm how this value is derived
                "bp_sequence": aligned_gen,
                "imgt_release": imgt_release
            }

            # Separate CSV file, GFE foreign key: a_name
            nuc_alignment = {
                "label": "NUC_ALIGN",
                "hla_name": hla_name,
                "a_name": a_name, # hla_name.split("-")[1]
                "length": len(aligned_nuc),
                "rank": "0", # TO DO: confirm how this value is derived
                "bp_sequence": aligned_nuc,
                "imgt_release": imgt_release
            }

            # Separate CSV file, GFE foreign key: a_name
            prot_alignment = {
                "label": "PROT_ALIGN",
                "hla_name": hla_name,
                "a_name": a_name, # hla_name.split("-")[1]
                "length": len(aligned_prot),
                "rank": "0", # TO DO: confirm how this value is derived
                "aa_sequence": aligned_prot,
                "imgt_release": imgt_release
            }


            # Separate CSV file, GFE foreign key: alleleId
            allele_groups = []
            
            for group in groups:
                group_dict = {
                    "alleleId": allele.id,
                    "hla_name": hla_name,
                    "a_name": a_name,
                    "ard_id": group[0],
                    "ard_name": group[1],
                    "locus": loc,
                    "imgt_release": imgt_release
                }

                allele_groups.append(group_dict)
                
            # Build CDS dict for CSV export, foreign key: alleleId, hla_name
            bp_seq, aa_seq = get_cds(allele)

            cds = {
                "alleleId": allele.id,
                "hla_name": hla_name,
                "bp_sequence": bp_seq,
                "aa_sequence": aa_seq,
                "imgt_release": imgt_release
            }
            
            # features preprocessing steps
            # 1) Convert seqann type to python dict using literal_eval
            # 2) add GFE foreign keys: alleleId, hla_name
            # 3) add columns: length

            # features contains list of seqann objects, converts to dict, destructive step
            features = [ast.literal_eval(str(feature).replace('\'', '"').replace('\n', '')) for feature in features]

            # Append allele id's
            # Note: Some alleles may have the same feature, but it may not be the same rank, 
            # so a feature should be identified with its allele by alleleId or HLA name
            for feature in features:
                feature["term"] = feature["term"].upper()
                feature["alleleId"] = allele.id 
                feature["hla_name"] = hla_name
                feature["imgt_release"] = imgt_release
                
                # Avoid null values in CSV for Neo4j import
                feature["hash_code"] = "none" if not feature["hash_code"] else feature["hash_code"]
                #feature["hla_name"] = "none" if not feature["hla_name"] else feature["hla_name"]
                
            # Append data to respective list
            data = zip(
                [gfe_sequences, gen_alignments, nuc_alignments, prot_alignments, all_cds],
                [gfe_sequence, gen_alignment, nuc_alignment, prot_alignment, cds]
            )
            
            for _list, _dict in data:
                _list.append(_dict)
                
            # Alignments, features, and ARD groups can all be concatenated since the keys are the same
            alignments = gen_alignments + nuc_alignments + prot_alignments
            all_features = all_features + features        
            all_groups = all_groups + allele_groups
        
        # Break point for testing
        if idx == 100:
                break
    
    # Remove duplicates
    tables = [gfe_sequences, alignments, all_features, all_groups, all_cds]
    files = ["gfe_sequences", "alignments", "features", "groups", "cds"]

    dataframes = []

    path = "../../data/csv/update/"

    # Output to CSV, include dbversion in name
    for table, file in zip(tables, files):
        dataframe = pd.DataFrame(table)
        file_name = path + file + f".{dbversion}.csv"
        dataframe.to_csv(file_name, index=False)
    
    

02/07/2021 07:07:38 AM - root - INFO - Loading ../bin/../../data/3360/A_gen.msf
02/07/2021 07:07:39 AM - root - INFO - Loaded 1771 genomic HLA-A sequences
02/07/2021 07:07:39 AM - root - INFO - Loading ../bin/../../data/3360/A_nuc.msf
02/07/2021 07:07:39 AM - root - INFO - Loaded 5016 nuc HLA-A sequences
02/07/2021 07:07:39 AM - root - INFO - Loading ../bin/../../data/3360/A_prot.msf
02/07/2021 07:07:40 AM - root - INFO - Loaded 5016 prot HLA-A sequences
02/07/2021 07:07:40 AM - root - INFO - Loading ../bin/../../data/3360/B_gen.msf
02/07/2021 07:07:40 AM - root - INFO - Loaded 2149 genomic HLA-B sequences
02/07/2021 07:07:40 AM - root - INFO - Loading ../bin/../../data/3360/B_nuc.msf
02/07/2021 07:07:41 AM - root - INFO - Loaded 6094 nuc HLA-B sequences
02/07/2021 07:07:41 AM - root - INFO - Loading ../bin/../../data/3360/B_prot.msf
02/07/2021 07:07:53 AM - root - INFO - Loaded 6094 prot HLA-B sequences
02/07/2021 07:07:53 AM - root - INFO - Loading ../bin/../../data/3360/C_gen.msf
02

02/07/2021 07:08:56 AM - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-1-0-159-0-0-0-0-0-0-0-0-0-0-0
02/07/2021 07:08:56 AM - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-134-0-1-0-0-0-0-0-0-0-0-0-0-0
02/07/2021 07:08:56 AM - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-135-0-1-0-0-0-0-0-0-0-0-0-0-0
02/07/2021 07:08:56 AM - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-1-0-160-0-0-0-0-0-0-0-0-0-0-0
02/07/2021 07:08:56 AM - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-136-0-1-0-0-0-0-0-0-0-0-0-0-0
02/07/2021 07:08:57 AM - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-1-0-161-0-0-0-0-0-0-0-0-0-0-0
02/07/2021 07:08:57 AM - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-1-0-162-0-1-0-0-0-0-0-0-0-0-0
02/07/2021 07:08:57 AM - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-1-0-163-0-0-0-0-0-0-0-0-0-0-0
02/07/2021 07:08:57 AM - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-137-0-1-0-0-0-0-0-0-0-0-0-0-0
02/07/2021 07:08:57 AM - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-138-0-1-0-0-0-0-0-0-0-0-0-0-0


In [20]:
allele.features

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(895), strand=1), type='source'),
 SeqFeature(CompoundLocation([FeatureLocation(BeforePosition(0), ExactPosition(73), strand=1), FeatureLocation(ExactPosition(73), ExactPosition(343), strand=1), FeatureLocation(ExactPosition(343), ExactPosition(619), strand=1), FeatureLocation(ExactPosition(619), ExactPosition(895), strand=1)], 'join'), type='CDS', location_operator='join'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(73), strand=1), type='exon'),
 SeqFeature(FeatureLocation(ExactPosition(73), ExactPosition(343), strand=1), type='exon'),
 SeqFeature(FeatureLocation(ExactPosition(343), ExactPosition(619), strand=1), type='exon'),
 SeqFeature(FeatureLocation(ExactPosition(619), ExactPosition(895), strand=1), type='exon')]

## CSV Validation

Each list of dictionaries is exported as A CSV. The keys of each dictionary represent columns of the CSV output. If the column names are changed, the Cypher script must also be updated.

Notes for testing:
- For each dataframe output as CSV, check that there are no NULL values in the dataframes, they should be replaced with "none" for convenient loading in Cypher

In [12]:
gfe_sequences

[{'alleleId': 'HLA00001.1',
  'gfe_name': 'HLA-Aw2-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-4',
  'locus': 'HLA-A',
  'hla_name': 'HLA-A*01:01:01:01',
  'a_name': 'A*01:01:01:01',
  'sequence': 'CAGGAGCAGAGGGGTCAGGGCGAAGTCCCAGGGCCCCAGGCGTGGCTCTCAGGGTCTCAGGCCCCGAAGGCGGTGTATGGATTGGGGAGTCCCAGCCTTGGGGATTCCCCAACTCCGCAGTTTCTTTTCTCCCTCTCCCAACCTACGTAGGGTCCTTCATCCTGGATACTCACGACGCGGACCCAGTTCTCACTCCCATTGGGTGTCGGGTTTCCAGAGAAGCCAATCAGTGTCGTCGCGGTCGCTGTTCTAAAGTCCGCACGCACCCACCGGGACTCAGATTCTCCCCAGACGCCGAGGATGGCCGTCATGGCGCCCCGAACCCTCCTCCTGCTACTCTCGGGGGCCCTGGCCCTGACCCAGACCTGGGCGGGTGAGTGCGGGGTCGGGAGGGAAACCGCCTCTGCGGGGAGAAGCAAGGGGCCCTCCTGGCGGGGGCGCAGGACCGGGGGAGCCGCGCCGGGAGGAGGGTCGGGCAGGTCTCAGCCACTGCTCGCCCCCAGGCTCCCACTCCATGAGGTATTTCTTCACATCCGTGTCCCGGCCCGGCCGCGGGGAGCCCCGCTTCATCGCCGTGGGCTACGTGGACGACACGCAGTTCGTGCGGTTCGACAGCGACGCCGCGAGCCAGAAGATGGAGCCGCGGGCGCCGTGGATAGAGCAGGAGGGGCCGGAGTATTGGGACCAGGAGACACGGAATATGAAGGCCCACTCACAGACTGACCGAGCGAACCTGGGGACCCTGCGCGGCTACTACAACCAGAGCGAGGACGGTGAGTGACCCCGGCCCGGGGCGCAGGTCACGACCCCTCATCC

In [13]:
alignments

[{'label': 'GEN_ALIGN',
  'hla_name': 'HLA-A*01:01:01:01',
  'a_name': 'A*01:01:01:01',
  'length': 3848,
  'rank': '0',
  'bp_sequence': 'CAGGAGCAGAGGGGTCAGGGCGAAGTCCCAGGGCCCCAGGCGTGGCTCTCAGGGTCTCAGGCCCCGAAGGCGGTGTATGGATTGGGGAGTCCCAGCCTTGGGGATTCCCCAACTCCGCAGTTTCTTTTCTCCCTCTCCCAACCTACGTAGGGTCCTTCATCCTGGATACTCACGACGCGGACCCAGTTCTCACTCCCATTGGGTGTCGGGTTTCCAGAGAAGCCAATCAGTGTCGTCGCGGTCGCTGTTCTAAAGTCCGCACGCACCCACCGGGACTCAGATTCTCCCC-AGACGCCGAGGATGGCCGTCATGGCGCCCCGAACCCTCCTCCTGCTACTCTCGGGGGCCCTGGCCCTGACCCAGACCTGGGCGGGTGAGTGCGGGGTCGGGAGGGAAACCGCCTCTGCGGGGAGAAGCAAGGGGCCCTCCTGGCGGGGGCGCAGGACCGGGGGAGCCGCGCCGGGAGGAGGGTCGGGCAGGTCTCAGCCACTGCTCGCCCCCAGGCTCCC-ACTCCATGAGGTATTTCTTCACATCCGTGTCCC-----GGCCCGGCCGCGGGGAGCC----CCGCTTCATCGCCGTGGGC-----------------------TACGTGGAC----GACACG-CAGTTCGTGCGGTTCGACAGCGACGCCGCGAGCCAGA--AGATGGAGCCG--------------------CGGGCGCCGTGGATAGAGCAGGAGGGGCCGGAGTATTGGGACCAGG--------------AGACACGGA-ATATGAAGGCCC-ACTCACAGACTGACCGAGCGAACCTGGGGACCCTGCGCGGCTACTACAACCAGAGCGAGGACGGTGAGTGACCC

In [14]:
all_features

[{'accession': 2,
  'hash_code': 'none',
  'locus': 'HLA-A',
  'rank': 1,
  'sequence': 'CAGGAGCAGAGGGGTCAGGGCGAAGTCCCAGGGCCCCAGGCGTGGCTCTCAGGGTCTCAGGCCCCGAAGGCGGTGTATGGATTGGGGAGTCCCAGCCTTGGGGATTCCCCAACTCCGCAGTTTCTTTTCTCCCTCTCCCAACCTACGTAGGGTCCTTCATCCTGGATACTCACGACGCGGACCCAGTTCTCACTCCCATTGGGTGTCGGGTTTCCAGAGAAGCCAATCAGTGTCGTCGCGGTCGCTGTTCTAAAGTCCGCACGCACCCACCGGGACTCAGATTCTCCCCAGACGCCGAGG',
  'term': 'FIVE_PRIME_UTR',
  'alleleId': 'HLA00001.1',
  'hla_name': 'HLA-A*01:01:01:01',
  'imgt_release': '3.36.0'},
 {'accession': 1,
  'hash_code': 'none',
  'locus': 'HLA-A',
  'rank': '1',
  'sequence': 'ATGGCCGTCATGGCGCCCCGAACCCTCCTCCTGCTACTCTCGGGGGCCCTGGCCCTGACCCAGACCTGGGCGG',
  'term': 'EXON',
  'alleleId': 'HLA00001.1',
  'hla_name': 'HLA-A*01:01:01:01',
  'imgt_release': '3.36.0'},
 {'accession': 1,
  'hash_code': 'none',
  'locus': 'HLA-A',
  'rank': '1',
  'sequence': 'GTGAGTGCGGGGTCGGGAGGGAAACCGCCTCTGCGGGGAGAAGCAAGGGGCCCTCCTGGCGGGGGCGCAGGACCGGGGGAGCCGCGCCGGGAGGAGGGTCGGGCAGGTCTCAGCCACTGC

In [15]:
all_groups

[{'alleleId': 'HLA00001.1',
  'a_name': 'A*01:01:01:01',
  'ard_id': 'HLA-A*01:01:01G',
  'ard_name': 'G',
  'locus': 'HLA-A',
  'imgt_release': '3.36.0'},
 {'alleleId': 'HLA00001.1',
  'a_name': 'A*01:01:01:01',
  'ard_id': 'HLA-A*01:01g',
  'ard_name': 'lg',
  'locus': 'HLA-A',
  'imgt_release': '3.36.0'},
 {'alleleId': 'HLA00001.1',
  'a_name': 'A*01:01:01:01',
  'ard_id': 'HLA-A*01:01',
  'ard_name': 'lgx',
  'locus': 'HLA-A',
  'imgt_release': '3.36.0'},
 {'alleleId': 'HLA00001.1',
  'a_name': 'A*01:01:01:01',
  'ard_id': 'A*01:01',
  'ard_name': '2nd_FIELD',
  'locus': 'HLA-A',
  'imgt_release': '3.36.0'},
 {'alleleId': 'HLA02169.1',
  'a_name': 'A*01:01:01:02N',
  'ard_id': 'HLA-A*01:01:01G',
  'ard_name': 'G',
  'locus': 'HLA-A',
  'imgt_release': '3.36.0'},
 {'alleleId': 'HLA02169.1',
  'a_name': 'A*01:01:01:02N',
  'ard_id': 'HLA-A*01:01g',
  'ard_name': 'lg',
  'locus': 'HLA-A',
  'imgt_release': '3.36.0'},
 {'alleleId': 'HLA02169.1',
  'a_name': 'A*01:01:01:02N',
  'ard_id'

In [16]:
all_cds

[{'alleleId': 'HLA00001.1',
  'hla_name': 'HLA-A*01:01:01:01',
  'bp_sequence': 'ATGGCCGTCATGGCGCCCCGAACCCTCCTCCTGCTACTCTCGGGGGCCCTGGCCCTGACCCAGACCTGGGCGGGCTCCCACTCCATGAGGTATTTCTTCACATCCGTGTCCCGGCCCGGCCGCGGGGAGCCCCGCTTCATCGCCGTGGGCTACGTGGACGACACGCAGTTCGTGCGGTTCGACAGCGACGCCGCGAGCCAGAAGATGGAGCCGCGGGCGCCGTGGATAGAGCAGGAGGGGCCGGAGTATTGGGACCAGGAGACACGGAATATGAAGGCCCACTCACAGACTGACCGAGCGAACCTGGGGACCCTGCGCGGCTACTACAACCAGAGCGAGGACGGTTCTCACACCATCCAGATAATGTATGGCTGCGACGTGGGGCCGGACGGGCGCTTCCTCCGCGGGTACCGGCAGGACGCCTACGACGGCAAGGATTACATCGCCCTGAACGAGGACCTGCGCTCTTGGACCGCGGCGGACATGGCAGCTCAGATCACCAAGCGCAAGTGGGAGGCGGTCCATGCGGCGGAGCAGCGGAGAGTCTACCTGGAGGGCCGGTGCGTGGACGGGCTCCGCAGATACCTGGAGAACGGGAAGGAGACGCTGCAGCGCACGGACCCCCCCAAGACACATATGACCCACCACCCCATCTCTGACCATGAGGCCACCCTGAGGTGCTGGGCCCTGGGCTTCTACCCTGCGGAGATCACACTGACCTGGCAGCGGGATGGGGAGGACCAGACCCAGGACACGGAGCTCGTGGAGACCAGGCCTGCAGGGGATGGAACCTTCCAGAAGTGGGCGGCTGTGGTGGTGCCTTCTGGAGAGGAGCAGAGATACACCTGCCATGTGCAGCATGAGGGTCTGCCCAAGCCCCTCACCCTGAGATGGGAGCTGTCTTCCCAGCCCACCATCC

## KIR

Not working due to error:
```python
ValueError: Problem with 'CDS' feature:
join(269..302,1267..1302,3751..4049,5579..5872,9027..9077,
13337..13438,13901..13953,14052..14228)
/codon_start=1
/gene="KIR2DL1"
/allele="KIR2DL1*0450101N"
/product="KIR2DL1 Killer-cell Immunoglobulin-like Receptor"
/translation="MSLLFVSMACVGFFLLQGAWPHEGVHRNLPSWPTQVPWX
```

In [20]:
# KIR process (broken)
if kir:
    if verbose:
        logging.info("Adding KIR to GFE DB")

    kir_file = data_dir + 'KIR.dat'

    if align:
        aligned = kir_alignments()

    # Downloading KIR
    if not os.path.isfile(kir_file):
        if verbose:
            logging.info("Downloading KIR dat file from " + kir_url)
        urllib.request.urlretrieve(kir_url, kir_file)

    kir_gen = SeqIO.parse(kir_file, "imgt")
    if verbose:
        logging.info("Finished parsing KIR dat file")

    i = 0
    for idx, allele in enumerate(kir_gen):
    
        # Breakpoint for development testing
        if idx == 1:
                break
        
        if hasattr(allele, 'seq'):
            loc = allele.description.split(",")[0].split("*")[0]
            if loc in kir_loci and len(str(allele.seq)) > 5:
                if debug:
                    logging.info(
                        "KIR = " + allele.description.split(",")[0] + " " + kir_release)

                groups = []
                complete_annotation = get_features(allele)
                ambigs = [
                    a for a in complete_annotation if re.search("/", a)]

                aligned_seq = ''
                if align:
                    if allele.description.split(",")[0] in aligned[loc]:
                        aligned_seq = aligned[loc][allele.description.split(",")[
                            0]]

                if ambigs:
                    logging.info(
                        "AMBIGS " + allele.description.split(",")[0] + " " + kir_release)
                    annotations = []
                    for ambig in ambigs:
                        if debug:
                            logging.info("AMBIG = " + ambig)
                        aterm = ambig.split("/")[0].split("_")[0]
                        anno = {
                            a: complete_annotation[a] for a in complete_annotation if a not in ambigs}
                        anno.update(
                            {ambig.split("/")[0]: complete_annotation[ambig]})
                        annotations.append(anno)

                        anno2 = {
                            a: complete_annotation[a] for a in complete_annotation if a not in ambigs}
                        anno2.update(
                            {aterm + "_" + ambig.split("/")[1]: complete_annotation[ambig]})
                        annotations.append(anno2)

                    for annotation in annotations:
                        ann = Annotation(annotation=annotation,
                                         method='match',
                                         complete_annotation=True)

                        features, gfe = gfe_maker.get_gfe(ann, loc)

01/30/2021 02:48:52 PM - root - INFO - Adding KIR to GFE DB
01/30/2021 02:48:53 PM - root - INFO - Finished parsing KIR dat file


ValueError: Problem with 'CDS' feature:
join(269..302,1267..1302,3751..4049,5579..5872,9027..9077,
13337..13438,13901..13953,14052..14228)
/codon_start=1
/gene="KIR2DL1"
/allele="KIR2DL1*0450101N"
/product="KIR2DL1 Killer-cell Immunoglobulin-like Receptor"
/translation="MSLLFVSMACVGFFLLQGAWPHEGVHRNLPSWPTQVPWX